In [14]:
import json
import cv2
import numpy as np

In [15]:
row, col = 0, 10
patch_size = 1024
overlap = 0.0625 
step_size = int(patch_size * (1 - overlap))  # 960 pixels
# step_size = 900

In [16]:
# Calculate patch offset
x_offset = col * step_size  # 9600
y_offset = row * step_size  # 0
patch_bounds = (x_offset, y_offset, x_offset + patch_size, y_offset + patch_size)  # (9600, 0, 10624, 1024)

In [17]:
# Load GeoJSON from full WSI inference
geojson_path = "/home/KutumLabGPU/Documents/santosh/capstone_project/models/CellViT/example/output/preprocessing/TCGA-V5-A7RE-11A-01-TS1.57401526-EF9E-49AC-8FF6-B4F9652311CE/cell_detection/cells.geojson"  # Update this
with open(geojson_path, 'r') as f:
    geojson_data = json.load(f)

In [18]:
# Function to calculate centroid of a polygon (first ring only for simplicity)
def get_centroid(coords):
    coords = np.array(coords)
    x, y = coords[:, 0], coords[:, 1]
    centroid_x = np.mean(x)
    centroid_y = np.mean(y)
    return centroid_x, centroid_y

In [19]:
# Filter and transform coordinates
patch_features = []
for feature in geojson_data:  # Directly iterate over the list
    if feature["geometry"]["type"] == "MultiPolygon":
        # Use the first polygon ring for centroid calculation
        coords = feature["geometry"]["coordinates"][0][0]  # First MultiPolygon, first ring
        centroid_x, centroid_y = get_centroid(coords)
        
        # Check if centroid is within patch bounds
        if (patch_bounds[0] <= centroid_x < patch_bounds[2] and 
            patch_bounds[1] <= centroid_y < patch_bounds[3]):
            # Transform all rings to patch-local coordinates
            local_coords = []
            for polygon in feature["geometry"]["coordinates"]:
                transformed_rings = []
                for ring in polygon:
                    transformed_ring = [[x - x_offset, y - y_offset] for x, y in ring]
                    transformed_rings.append(transformed_ring)
                local_coords.append(transformed_rings)
            feature["geometry"]["coordinates"] = local_coords  # Update coordinates
            patch_features.append(feature)

In [20]:
# Save filtered GeoJSON (optional)
patch_geojson = patch_features  # Already a list, no need for FeatureCollection wrapper
with open("patch_segmentation.geojson", "w") as f:
    json.dump(patch_geojson, f)

In [21]:
# Load patch image
patch_image_path = "/home/KutumLabGPU/Documents/santosh/capstone_project/models/snake_algo_implement/TCGA-WSI-patch.png"
patch = cv2.imread(patch_image_path)

In [22]:
print(f"Patch image shape: {patch.shape}")

Patch image shape: (1024, 1024, 3)


In [23]:
# Overlay nuclei
for feature in patch_features:
    for polygon in feature["geometry"]["coordinates"]:  # Iterate over MultiPolygon
        coords = np.array(polygon[0], dtype=np.int32)  # First ring of each polygon
        # x offset by +33
        coords[:, 0] += 33
        coords[:, 1] += 33
        # set polygone color to red
        cv2.polylines(patch, [coords], isClosed=True, color=(0, 0, 255), thickness=2)
        # cv2.polylines(patch, [coords], color=(0, 0, 255), thickness=1)


# Save or display the result
cv2.imwrite("patch_with_nuclei.png", patch)
# cv2.imshow("Patch with Nuclei", patch)

True